In [1]:
import numpy as np
import numba
from numba import jit, prange
import matplotlib.pyplot as plt

In [2]:
# Cell 2: Agent Initialization and Tax Calculation
@jit(nopython=True)
def initialize_agents(num_agents):
    agents = np.zeros(num_agents, dtype=np.float32)
    
    n_poor = int(0.5 * num_agents)
    n_middle = int(0.4 * num_agents)
    n_rich = num_agents - n_poor - n_middle
    
    agents[:n_poor] = np.random.uniform(0.1, 3, n_poor)
    agents[n_poor:n_poor+n_middle] = np.random.uniform(3, 12, n_middle)
    agents[n_poor+n_middle:] = np.random.uniform(12, 50, n_rich)
    
    np.random.shuffle(agents)
    return agents

@jit(nopython=True)
def calculate_tax(wealth):
    slabs = np.array([3, 6, 9, 12, 15], dtype=np.float32)
    rates = np.array([0, 0.05, 0.1, 0.15, 0.2, 0.3], dtype=np.float32)
    
    tax = 0.0
    for i in range(len(slabs)):
        if wealth <= slabs[i]:
            tax += rates[i] * (wealth - (slabs[i-1] if i > 0 else 0))
            break
        else:
            tax += rates[i] * (slabs[i] - (slabs[i-1] if i > 0 else 0))
    
    if wealth > slabs[-1]:
        tax += rates[-1] * (wealth - slabs[-1])
    
    return tax

In [3]:
# Cell 3: Trade Probability and Agent Update Functions
@jit(nopython=True)
def calculate_trade_prob(agent1_wealth, agent2_wealth):
    return np.exp(-np.abs(agent1_wealth - agent2_wealth))

@jit(nopython=True)
def initialize_trade_probabilities(agents):
    num_agents = len(agents)
    trade_probs = np.zeros((num_agents, num_agents), dtype=np.float32)
    for i in range(num_agents):
        for j in range(i + 1, num_agents):
            prob = calculate_trade_prob(agents[i], agents[j])
            trade_probs[i, j] = prob
            trade_probs[j, i] = prob
    return trade_probs

@jit(nopython=True)
def roulette_wheel_selection(probs):
    r = np.random.random()
    cum_prob = 0.0
    for i, prob in enumerate(probs):
        cum_prob += prob
        if r <= cum_prob:
            return i
    return len(probs) - 1

@jit(nopython=True)
def update_agents(agents, trade_probs):
    num_agents = len(agents)
    agent1_index = np.random.randint(0, num_agents)
    agent2_index = roulette_wheel_selection(trade_probs[agent1_index])
    
    transaction_amount = 0.075 * (agents[agent1_index] * agents[agent2_index]) / (agents[agent1_index] + agents[agent2_index])
    agents[agent1_index] -= transaction_amount
    agents[agent2_index] += transaction_amount
    
    for i in range(num_agents):
        if i != agent1_index:
            prob = calculate_trade_prob(agents[agent1_index], agents[i])
            trade_probs[agent1_index, i] = prob
            trade_probs[i, agent1_index] = prob
        if i != agent2_index:
            prob = calculate_trade_prob(agents[agent2_index], agents[i])
            trade_probs[agent2_index, i] = prob
            trade_probs[i, agent2_index] = prob

@jit(nopython=True)
def business_venture(agents, venture_probability):
    num_agents = len(agents)
    for i in range(num_agents):
        if np.random.random() < venture_probability:
            # 50% chance of success, 50% chance of failure
            if np.random.random() < 0.5:
                # Success: gain 20% of current wealth
                agents[i] *= 1.2
            else:
                # Failure: lose 20% of current wealth
                agents[i] *= 0.8
    return agents

In [4]:
# Cell 4: Simulation Function
@jit(nopython=True)
def run_simulation(num_agents, num_years, num_time_steps, venture_probability):
    agents = initialize_agents(num_agents)
    trade_probs = initialize_trade_probabilities(agents)
    
    # Calculate number of years based on num_time_steps
    transactions_per_year = num_time_steps
    num_years = num_years
    
    gini_indices = np.zeros(num_years + 1, dtype=np.float32)
    gini_indices[0] = gini_coefficient(agents)
    
    # Track specific agents
    rich_index = np.argmax(agents)
    poor_index = np.argmin(agents)
    middle_index = np.argsort(agents)[num_agents // 2]
    
    rich_wealth = np.zeros(num_years + 1, dtype=np.float32)
    middle_wealth = np.zeros(num_years + 1, dtype=np.float32)
    poor_wealth = np.zeros(num_years + 1, dtype=np.float32)
    
    rich_wealth[0] = agents[rich_index]
    middle_wealth[0] = agents[middle_index]
    poor_wealth[0] = agents[poor_index]
    
    total_wealth = np.sum(agents)
    
    for step in range(num_time_steps):
        update_agents(agents, trade_probs)
        agents = business_venture(agents, venture_probability)
        
        # Deduct tax
        total_tax = 0.0
        for i in range(num_agents):
            tax = calculate_tax(agents[i])
            agents[i] -= tax
            total_tax += tax
        
        # Redistribute tax
        redistribution = total_tax / num_agents
        agents += redistribution
        
        # Annual updates
        if (step + 1) % transactions_per_year == 0:
            year = (step + 1) // transactions_per_year
            gini_indices[year] = gini_coefficient(agents)
            rich_wealth[year] = agents[rich_index]
            middle_wealth[year] = agents[middle_index]
            poor_wealth[year] = agents[poor_index]
            
            # Check for wealth conservation
            current_total_wealth = np.sum(agents)
            if not np.isclose(total_wealth, current_total_wealth, rtol=1e-5):
                print(f"Warning: Total wealth changed in year {year}")
                print("Initial:", total_wealth, "Current:", current_total_wealth)
            
            # Print wealth distribution statistics
            print(f"Year {year}: Min wealth: {np.min(agents):.2f}, Max wealth: {np.max(agents):.2f}, Mean wealth: {np.mean(agents):.2f}")
    
    return agents, gini_indices, rich_wealth, middle_wealth, poor_wealth

@jit(nopython=True)
def gini_coefficient(wealths):
    sorted_wealths = np.sort(wealths)
    n = len(sorted_wealths)
    index = np.arange(1, n + 1)
    return (np.sum((2 * index - n - 1) * sorted_wealths)) / (n * np.sum(sorted_wealths))

In [ ]:
# Cell 5: Run Simulation
# Simulation parameters
num_agents = 10000
num_years = 50
num_time_steps = 10000  #per year
venture_probability = 0.4 #percentage of people starting a business

# Run the simulation
final_wealths, gini_indices, rich_wealth, middle_wealth, poor_wealth = run_simulation(num_agents, num_years, num_time_steps, venture_probability)

print(f"Final Gini coefficient: {gini_indices[-1]:.4f}")

Initial: 68663.3203125 Current: 68705.921875
Step 1000 : Min wealth: 2.3029773235321045 , Max wealth: 15.171504020690918 , Mean wealth: 6.8705921875
Initial: 68663.3203125 Current: 65764.078125
Step 2000 : Min wealth: 1.9656801223754883 , Max wealth: 15.18543815612793 , Mean wealth: 6.5764078125
Initial: 68663.3203125 Current: 65033.8203125
Step 3000 : Min wealth: 2.310462474822998 , Max wealth: 15.352862358093262 , Mean wealth: 6.50338203125
Initial: 68663.3203125 Current: 60425.4375
Step 4000 : Min wealth: 1.9904204607009888 , Max wealth: 14.459758758544922 , Mean wealth: 6.04254375
Initial: 68663.3203125 Current: 62189.296875
Step 5000 : Min wealth: 1.8551814556121826 , Max wealth: 14.477211952209473 , Mean wealth: 6.2189296875
Initial: 68663.3203125 Current: 60736.78125
Step 6000 : Min wealth: 1.7306374311447144 , Max wealth: 14.701447486877441 , Mean wealth: 6.073678125
Initial: 68663.3203125 Current: 58208.67578125
Step 7000 : Min wealth: 1.5654853582382202 , Max wealth: 15.52941

In [ ]:
# Cell 6: Visualize Gini Coefficient Over Time
plt.figure(figsize=(10, 6))
plt.figure(figsize=(10, 6))
years = np.arange(num_years)
plt.plot(years, gini_indices)
plt.title('Gini Coefficient Over Time')
plt.xlabel('Years')
plt.ylabel('Gini Coefficient')
plt.grid(True)
plt.show()

In [ ]:
# Cell 7: Visualize Final Wealth Distribution
plt.figure(figsize=(10, 6))
plt.hist(final_wealths, bins=50, edgecolor='black')
plt.title('Final Wealth Distribution')
plt.xlabel('Wealth')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Cell 8: Visualize Wealth of Rich, Middle, and Poor Agents
plt.figure(figsize=(10, 6))
plt.plot(years, rich_wealth, label='Rich Agent')
plt.plot(years, middle_wealth, label='Middle Agent')
plt.plot(years, poor_wealth, label='Poor Agent')
plt.title('Wealth of Specific Agents Over Time')
plt.xlabel('Years')
plt.ylabel('Wealth')
plt.legend()
plt.grid(True)
plt.show()